In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.datasets import cifar10
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from sklearn.model_selection import train_test_split
import pandas as pd
import optuna
import optuna.visualization as vis

In [2]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.cifar10.load_data()

X_train = X_train / 255
X_test = X_test / 255

X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train,
                                                       test_size=0.2, random_state=0)

classes = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

def show_image(dataset, labels, index):
    plt.figure(figsize=(15, 2))
    plt.imshow(dataset[index])
    plt.xlabel(classes[labels[index][0]])


170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step


In [3]:
def objective(trial):
    model = keras.Sequential()
    
    num_conv_layers = trial.suggest_int('num_conv_layers', 2, 5)
    
    for i in range(num_conv_layers):
        filters = trial.suggest_categorical(f'filters_layer_{i}', [32, 64, 128, 256, 512, 1024])
        model.add(layers.Conv2D(filters=filters, kernel_size=3, padding='same'))
        model.add(layers.BatchNormalization())
        model.add(layers.ReLU())
        model.add(layers.MaxPool2D(pool_size=(2, 2)))
        model.add(layers.Dropout(trial.suggest_float(f'dropout_layer_{i}', .2, .5)))
        
    model.add(layers.Flatten())
                  
    num_dense_neurons = trial.suggest_int('num_dense_neurons', 64, 512)
    model.add(layers.Dense(num_dense_neurons, activation='relu'))
    model.add(layers.Dropout(trial.suggest_float(f'dense_dropout_{i}', .2, .5)))
    model.add(layers.Dense(10, activation='softmax'))
    
    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['sparse_categorical_accuracy']
    )
                  
    early_stopping = EarlyStopping(
        patience=5,
        min_delta=.001,
        restore_best_weights=True
    )
    
    history = model.fit(
        X_train, y_train,
        validation_data=(X_valid, y_valid),
        callbacks=[early_stopping],
        epochs=100
    )
                  
                  
    score = model.evaluate(X_valid, y_valid)
                  
    return score[1]

In [4]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=20)
print(study.best_params)

[I 2024-09-19 18:11:17,732] A new study created in memory with name: no-name-da1674f1-cb44-4720-a6a5-b71b66c3cdac


Epoch 1/100


I0000 00:00:1726769483.072294      85 service.cc:145] XLA service 0x7afd380059d0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1726769483.072349      85 service.cc:153]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0


   6/1250 ━━━━━━━━━━━━━━━━━━━━ 30s 24ms/step - loss: 47.9577 - sparse_categorical_accuracy: 0.0922

I0000 00:00:1726769490.383800      85 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


...

Epoch 1/100
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 23s 8ms/step - loss: 1.9313 - sparse_categorical_accuracy: 0.3098 - val_loss: 1.3590 - val_sparse_categorical_accuracy: 0.5082
Epoch 2/100
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 9s 7ms/step - loss: 1.3732 - sparse_categorical_accuracy: 0.5003 - val_loss: 1.1402 - val_sparse_categorical_accuracy: 0.5950
Epoch 3/100
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 10s 8ms/step - loss: 1.1910 - sparse_categorical_accuracy: 0.5727 - val_loss: 1.2505 - val_sparse_categorical_accuracy: 0.5662
Epoch 4/100
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 9s 7ms/step - loss: 1.0741 - sparse_categorical_accuracy: 0.6222 - val_loss: 1.0117 - val_sparse_categorical_accuracy: 0.6367
Epoch 5/100
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 9s 7ms/step - loss: 0.9823 - sparse_categorical_accuracy: 0.6560 - val_loss: 1.1095 - val_sparse_categorical_accuracy: 0.6241
Epoch 6/100
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 9s 7ms/step - loss: 0.8949 - sparse_categorical_accuracy: 0.6914 - val_loss: 1.3142 - val_sparse_categorical_a

[I 2024-09-19 19:58:47,571] Trial 18 finished with value: 0.7402999997138977 and parameters: {'num_conv_layers': 4, 'filters_layer_0': 256, 'dropout_layer_0': 0.3375265820502196, 'filters_layer_1': 64, 'dropout_layer_1': 0.30783630673494095, 'filters_layer_2': 1024, 'dropout_layer_2': 0.3546547561833453, 'filters_layer_3': 128, 'dropout_layer_3': 0.3116590867110244, 'num_dense_neurons': 272, 'dense_dropout_3': 0.4978398933328384}. Best is trial 11 with value: 0.8238999843597412.


Epoch 1/100
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 46s 29ms/step - loss: 2.5479 - sparse_categorical_accuracy: 0.2954 - val_loss: 1.5503 - val_sparse_categorical_accuracy: 0.4501
Epoch 2/100
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 35s 28ms/step - loss: 1.3874 - sparse_categorical_accuracy: 0.4902 - val_loss: 1.9442 - val_sparse_categorical_accuracy: 0.4184
Epoch 3/100
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 35s 28ms/step - loss: 1.2280 - sparse_categorical_accuracy: 0.5535 - val_loss: 1.4612 - val_sparse_categorical_accuracy: 0.5039
Epoch 4/100
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 35s 28ms/step - loss: 1.1101 - sparse_categorical_accuracy: 0.5957 - val_loss: 1.1550 - val_sparse_categorical_accuracy: 0.5922
Epoch 5/100
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 35s 28ms/step - loss: 1.0247 - sparse_categorical_accuracy: 0.6328 - val_loss: 0.9810 - val_sparse_categorical_accuracy: 0.6613
Epoch 6/100
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 35s 28ms/step - loss: 0.9554 - sparse_categorical_accuracy: 0.6541 - val_loss: 1.1125 - val_sparse_cat

[I 2024-09-19 20:11:45,243] Trial 19 finished with value: 0.8097000122070312 and parameters: {'num_conv_layers': 3, 'filters_layer_0': 512, 'dropout_layer_0': 0.24635652307756895, 'filters_layer_1': 1024, 'dropout_layer_1': 0.39448888567142126, 'filters_layer_2': 512, 'dropout_layer_2': 0.4601713329531733, 'num_dense_neurons': 448, 'dense_dropout_2': 0.20007773441465637}. Best is trial 11 with value: 0.8238999843597412.


{'num_conv_layers': 4, 'filters_layer_0': 512, 'dropout_layer_0': 0.2924515218223497, 'filters_layer_1': 1024, 'dropout_layer_1': 0.20420635555557912, 'filters_layer_2': 1024, 'dropout_layer_2': 0.4938435933532547, 'filters_layer_3': 512, 'dropout_layer_3': 0.477406309189756, 'num_dense_neurons': 342, 'dense_dropout_3': 0.49155865824030837}


In [5]:
print(study.best_params)

{'num_conv_layers': 4, 'filters_layer_0': 512, 'dropout_layer_0': 0.2924515218223497, 'filters_layer_1': 1024, 'dropout_layer_1': 0.20420635555557912, 'filters_layer_2': 1024, 'dropout_layer_2': 0.4938435933532547, 'filters_layer_3': 512, 'dropout_layer_3': 0.477406309189756, 'num_dense_neurons': 342, 'dense_dropout_3': 0.49155865824030837}


In [12]:
def build_best_model(best_params):
    model = keras.Sequential()
    
    for i in range(best_params['num_conv_layers']):
        filters=best_params[f'filters_layer_{i}']
        model.add(layers.Conv2D(filters, kernel_size=3, padding='same'))
        model.add(layers.BatchNormalization())
        model.add(layers.ReLU())
        model.add(layers.MaxPooling2D(pool_size=(2, 2)))
        model.add(layers.Dropout(best_params[f'dropout_layer_{i}']))
    
    
    model.add(layers.Flatten())
    model.add(layers.Dense(best_params[f'num_dense_neurons'], activation='relu'))
    model.add(layers.Dropout(best_params[f'dense_dropout_{i}']))
    model.add(layers.Dense(10, activation='softmax'))
    
    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['sparse_categorical_accuracy']
    )
              
    return model

In [13]:
early_stopping = EarlyStopping(
    patience=5,
    min_delta=.001,
    restore_best_weights=True
)

best_model = build_best_model(study.best_params)
best_model.fit(X_train, y_train, 
               validation_data=(X_valid, y_valid),
               callbacks=[early_stopping],
               epochs=100
              )

Epoch 1/100
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 56s 36ms/step - loss: 2.1108 - sparse_categorical_accuracy: 0.2548 - val_loss: 1.6586 - val_sparse_categorical_accuracy: 0.3808
Epoch 2/100
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 44s 35ms/step - loss: 1.5584 - sparse_categorical_accuracy: 0.4195 - val_loss: 1.2902 - val_sparse_categorical_accuracy: 0.5442
Epoch 3/100
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 44s 35ms/step - loss: 1.3617 - sparse_categorical_accuracy: 0.5007 - val_loss: 1.2368 - val_sparse_categorical_accuracy: 0.5556
Epoch 4/100
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 44s 35ms/step - loss: 1.2332 - sparse_categorical_accuracy: 0.5588 - val_loss: 1.1962 - val_sparse_categorical_accuracy: 0.5679
Epoch 5/100
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 44s 35ms/step - loss: 1.1486 - sparse_categorical_accuracy: 0.5923 - val_loss: 1.2752 - val_sparse_categorical_accuracy: 0.5462
Epoch 6/100
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 44s 35ms/step - loss: 1.0595 - sparse_categorical_accuracy: 0.6237 - val_loss: 1.1940 - val_sparse_cat

In [15]:
loss, accuracy = best_model.evaluate(X_test, y_test)
print(f'loss: {loss}')
print(f'accuracy: {accuracy}')

313/313 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step - loss: 0.5988 - sparse_categorical_accuracy: 0.8172
loss: 0.5892660617828369
accuracy: 0.8147000074386597


In [ ]:
vis.plot_optimization_history(study)
vis.plot_param_importances(study)
vis.plot_slice(study)
vis.plot_parallel_coordinates(study)
vis.plot_contour(study)